In [1]:
import json
with open('./train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [2]:
len(data)

1000

In [3]:
data[0]

{'id': '213e438b17282904772446704d0b28',
 'instruction': '你是一个电商客服专家，请根据用户与客服的多轮对话判断用户的意图分类标签。\n<用户与客服的对话 START>\n用户: <image>\n客服: 抱歉哦，小富目前还不能识别图片，您可以简单描述一下遇到的问题吗？/:068~~\n用户: <http>\n客服: 若您仍有疑问，可回复“客服”以便进一步协助您~\n用户: 内胆使用的是316不锈钢吗\n客服: 亲爱的，升级版的抗菌不锈钢是在304不锈钢的基础上改进的，它有抗菌功能（其内胆表面能够释放具有抑菌作用的铜离子，有效阻止大肠杆菌和金黄色葡萄球菌等有害细菌的生长）；不过它的抗酸碱和抗腐蚀性能比不上316不锈钢；而316不锈钢在抗腐蚀和抗酸碱方面表现更佳，但没有抗菌功能。  ▲对于居住在海边或喜欢储存各种饮品的人来说，推荐使用316不锈钢；如果是为了儿童使用，建议选择抗菌不锈钢；追求性价比的话，304不锈钢是不错的选择；/:810\n<用户与客服的对话 END>\n请直接只输出分类标签结果，不需要其他多余的话。以下是可以参考的分类标签为：["反馈密封性不好","是否好用","是否会生锈","排水方式","包装区别","发货数量","反馈用后症状","商品材质","功效功能","是否易褪色","适用季节","能否调光","版本款型区别","单品推荐","用法用量","控制方式","上市时间","商品规格","信号情况","养护方法","套装推荐","何时上货","气泡"]\n',
 'input': '',
 'output': '商品材质',
 'image': ['213e438b17282904772446704d0b28-0.jpg']}

In [4]:
data_scene = []
data_dialogue = []
for dict in data:
    instruction = dict['instruction']
    if '你是一个电商领域识图专家,可以理解消费者上传的软件截图或实物拍摄图。' in instruction:
        data_scene.append(dict)
    elif '你是一个电商客服专家，请根据用户与客服的多轮对话判断用户的意图分类标签。' in instruction:
        data_dialogue.append(dict)

In [ ]:
len(data_scene), len(data_dialogue)
# scene训练数据大小是测试数据的 700/5000
# dialogue训练数据大小是测试数据的 300/5000

(700, 300)

In [ ]:
# 场景分类任务训练数据都只有一张图像
image_lengths = [len(dict["image"]) for dict in data_scene]
image_lengths_distribution = {length: image_lengths.count(length) for length in set(image_lengths)}
image_lengths_distribution

{1: 700}

In [6]:
# 对话分类任务有少部分多图情况，也有一个无图情况
image_lengths = [len(dict["image"]) for dict in data_dialogue]
image_lengths_distribution = {length: image_lengths.count(length) for length in set(image_lengths)}
image_lengths_distribution

{0: 1, 1: 266, 2: 29, 3: 3, 4: 1}

In [7]:
# 两类instruction举例
print(f"*场景分类任务*\n{data_scene[0]['instruction']}")
print(f"*对话分类任务*\n{data_dialogue[0]['instruction']}")
# 发现场景分类任务instruction没有有效信息，对话分类任务instruction有有效对话信息

*场景分类任务*
Picture 1: <image>
你是一个电商领域识图专家,可以理解消费者上传的软件截图或实物拍摄图。现在,请你对消费者上传的图片进行分类。你只需要回答图片分类结果,不需要其他多余的话。以下是可以参考的分类标签,分类标签:["实物拍摄(含售后)","商品分类选项","商品头图","商品详情页截图","下单过程中出现异常（显示购买失败浮窗）","订单详情页面","支付页面","消费者与客服聊天页面","评论区截图页面","物流页面-物流列表页面","物流页面-物流跟踪页面","物流页面-物流异常页面","退款页面","退货页面","换货页面","购物车页面","店铺页面","活动页面","优惠券领取页面","账单/账户页面","个人信息页面","投诉举报页面","平台介入页面","外部APP截图","其他类别图片"]。
*对话分类任务*
你是一个电商客服专家，请根据用户与客服的多轮对话判断用户的意图分类标签。
<用户与客服的对话 START>
用户: <image>
客服: 抱歉哦，小富目前还不能识别图片，您可以简单描述一下遇到的问题吗？/:068~~
用户: <http>
客服: 若您仍有疑问，可回复“客服”以便进一步协助您~
用户: 内胆使用的是316不锈钢吗
客服: 亲爱的，升级版的抗菌不锈钢是在304不锈钢的基础上改进的，它有抗菌功能（其内胆表面能够释放具有抑菌作用的铜离子，有效阻止大肠杆菌和金黄色葡萄球菌等有害细菌的生长）；不过它的抗酸碱和抗腐蚀性能比不上316不锈钢；而316不锈钢在抗腐蚀和抗酸碱方面表现更佳，但没有抗菌功能。  ▲对于居住在海边或喜欢储存各种饮品的人来说，推荐使用316不锈钢；如果是为了儿童使用，建议选择抗菌不锈钢；追求性价比的话，304不锈钢是不错的选择；/:810
<用户与客服的对话 END>
请直接只输出分类标签结果，不需要其他多余的话。以下是可以参考的分类标签为：["反馈密封性不好","是否好用","是否会生锈","排水方式","包装区别","发货数量","反馈用后症状","商品材质","功效功能","是否易褪色","适用季节","能否调光","版本款型区别","单品推荐","用法用量","控制方式","上市时间","商品规格","信号情况","养护方法","套装推荐","何时上货","气泡"]



In [8]:
### 对话任务label分布
label_str = data_dialogue[0]['instruction'].split('标签为：')[1].strip()
label = json.loads(label_str)
label_dict = {key: 0 for key in label}
for dict in data_dialogue:
    output = dict['output']
    label_dict[output] += 1
label_dict

{'反馈密封性不好': 15,
 '是否好用': 8,
 '是否会生锈': 8,
 '排水方式': 9,
 '包装区别': 17,
 '发货数量': 14,
 '反馈用后症状': 4,
 '商品材质': 14,
 '功效功能': 17,
 '是否易褪色': 4,
 '适用季节': 24,
 '能否调光': 14,
 '版本款型区别': 18,
 '单品推荐': 20,
 '用法用量': 17,
 '控制方式': 12,
 '上市时间': 11,
 '商品规格': 13,
 '信号情况': 9,
 '养护方法': 10,
 '套装推荐': 17,
 '何时上货': 13,
 '气泡': 12}

In [9]:
### 场景任务label分布
label_str = data_scene[0]['instruction'].split('分类标签:')[1].strip("。")
label = json.loads(label_str)
label_dict = {key: 0 for key in label}
for dict in data_scene:
    output = dict['output']
    label_dict[output] += 1
label_dict

{'实物拍摄(含售后)': 23,
 '商品分类选项': 39,
 '商品头图': 37,
 '商品详情页截图': 38,
 '下单过程中出现异常（显示购买失败浮窗）': 28,
 '订单详情页面': 15,
 '支付页面': 37,
 '消费者与客服聊天页面': 0,
 '评论区截图页面': 27,
 '物流页面-物流列表页面': 15,
 '物流页面-物流跟踪页面': 26,
 '物流页面-物流异常页面': 25,
 '退款页面': 36,
 '退货页面': 37,
 '换货页面': 23,
 '购物车页面': 37,
 '店铺页面': 30,
 '活动页面': 38,
 '优惠券领取页面': 38,
 '账单/账户页面': 36,
 '个人信息页面': 0,
 '投诉举报页面': 31,
 '平台介入页面': 18,
 '外部APP截图': 31,
 '其他类别图片': 35}